In [ ]:
import h5py
import pandas as pd
import numpy as np
from imdb import IMDb
from imdb._exceptions import IMDbError
import sqlite3

In [ ]:
def read_hdf5_file(file_path,dataset_name='similarity_matrix3'):
    with h5py.File(file_path,'r')as f:
        data=f[dataset_name][:]
    return data
file_path='similarity_matrix3.h5'
similarity_matrix=read_hdf5_file(file_path)
similarity_matrix=pd.DataFrame(similarity_matrix)
similarity_matrix.shape

In [ ]:
similarity_matrix.head()


In [ ]:
def evaluate(row):
    top_indices = row.nlargest(21).index  # Get the indices of the top 21 values
    result = pd.Series(np.nan, index=row.index)  # Initialize a Series with NaN
    result[top_indices] = row[top_indices]  # Assign the top 21 values to their original positions
    return result

In [ ]:
def compute(similarity_matrix, batch_size=1000, output_file='similarity_preprocess5.h5'):
    num_samples = similarity_matrix.shape[0]
    
    with h5py.File(output_file, 'w') as f:
        # Create an empty dataset in HDF5 to store the processed results
        dset = f.create_dataset(
            "similarity_preprocess5",
            shape=(num_samples, similarity_matrix.shape[1]),
            dtype=np.float32
        )

        for i in range(0, num_samples, batch_size):
            # Get the current batch of rows from similarity_matrix
            batch = similarity_matrix.iloc[i:i+batch_size]
            
            # Apply the 'evaluate' function to each row in the batch
            batch_result = batch.apply(lambda x: evaluate(x), axis=1)

            
            # Fill the HDF5 dataset with the processed batch
            dset[i:i+batch_size, :] = batch_result.values
            
            print(f"Processed batch {i} to {i+batch_size}")

    print(f"Data successfully saved to {output_file}")
compute(similarity_matrix, batch_size=1000, output_file='similarity_preprocess5.h5')



In [ ]:
similarity_matrix

In [ ]:
similarity_matrix.head()

In [ ]:
similarity_matrix.shape


In [ ]:
similarity_matrix

In [ ]:
def cal(value,col_idx):
    return col_idx if not pd.isna(value) else value

In [ ]:
def process_similarity_matrix_chunks(similarity_matrix, chunk_size):
    """
    Process a large similarity matrix in chunks for the custom cal function.
    """
    n_rows = similarity_matrix.shape[0]
    processed_chunks = []

    for start_row in range(0, n_rows, chunk_size):
        end_row = min(start_row + chunk_size, n_rows)
        chunk = similarity_matrix.iloc[start_row:end_row]
        
        # Apply the transformation for each row in the chunk
        processed_chunk = chunk.apply(
            lambda row: [cal(val, idx) for idx, val in enumerate(row)], axis=1
        )
        
        processed_chunks.append(processed_chunk)
        print(f"Processed rows {start_row} to {end_row-1}")

    # Combine all processed chunks into the final DataFrame
    result = pd.concat(processed_chunks, axis=0)
    return result

# Example usage:
# Assuming similarity_matrix is your large DataFrame
chunk_size = 1000  # Adjust based on memory capacity
similarity_matrix1 = process_similarity_matrix_chunks(similarity_matrix, chunk_size)

# Now similarity_matrix_processed contains the final result


In [ ]:
similarity_matrix1.head()

In [ ]:
similarity_matrix1.shape

In [ ]:
similarity_matrix= pd.DataFrame(similarity_matrix1.tolist())

In [ ]:
similarity_matrix.head()

In [ ]:
df_sorted = similarity_matrix.apply(lambda row: pd.Series([x for x in row if not pd.isna(x)] + [np.nan] * row.isna().sum()), axis=1)

In [ ]:
df_sorted.head()

In [ ]:
df_cleaned = df_sorted.dropna(axis=1, how='any')

In [ ]:
df_cleaned.head()

In [ ]:
df_cleaned.to_csv("data.csv")

In [ ]:
df=pd.read_csv('movie-name_data.csv')

In [ ]:
new_df=pd.read_csv('movies-with_posters.csv')

In [ ]:
df.head(50000)

In [ ]:
new_df.head(10)

In [ ]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
stop_words = set(stopwords.words('english'))

# Function to clean movie names
def clean_movie_name(name):
    # Remove special characters and convert to lowercase
    name = re.sub(r'[^a-zA-Z0-9\s]', '', name).lower()
    # Split the name into words and remove stop words
    name = ' '.join(word for word in name.split() if word not in stop_words)
    # Remove spaces to make it a continuous string
    return name.replace(" ", "")

# Apply the cleaning function to the movie_name column
df['movie_name'] = df['movie_name'].apply(clean_movie_name)

In [ ]:
df.head()

In [ ]:
#Ant-Man and the Wasp: Quantumania
#Spider-Man
#The Exorcist
#Avatar
#Avatar: The Way of Water
#RRR
#The Red Pearl

In [ ]:

movie_name="avatar"
movie_name=clean_movie_name(movie_name)
print("The movie name entered :",movie_name)
row_index=df.index[df['movie_name']==movie_name][0]
print(row_index)
list1= df_cleaned.iloc[row_index].tolist()
print(list1)
print("Top 20 recommended Movies : ")
for i in list1 :
    if i!=row_index:
      print(new_df.loc[i,'movie_name'])